In [1]:
from datetime import datetime, date, timedelta
import numpy as np
import xarray as xr
import sys
sys.path.append('/home/ascherrmann/scripts/')
import helper
import argparse
import pickle
import os

---
## Create File with cyclone information of case study 17 

In [4]:
MOT = "APR17"

path_trackCYC = f'/net/helium/atmosdyn/freimax/data_msc/IFS-17/trajectories/caltra/TRACKED_CYCLONE.txt'
d = np.loadtxt(path_trackCYC,skiprows=1)
try:
    IDs = np.unique(d[:,1])
except IndexError:
    IDs = np.unique(d[-1])
print(f'IDs from the TRACKED_CYCLONE file:\n{IDs}')


# ## Load validation cyclone IDs
# path_CYCsplit_info = f'/net/helium/atmosdyn/freimax/data_msc/IFS-18/cyclones/data_random_forest/{MOT}/cyclone_split_info.txt'
# with open(path_CYCsplit_info, 'r') as f:
#     lines = f.readlines()


# num_cyclones = int(lines[0].split(': ')[1])  # The 'Number of cyclones' line
# id_list_str = lines[1].split(': ')[1].strip('[]\n')
# id_list_cyc = [float(x) for x in id_list_str.split(', ')] if id_list_str else []  # The 'ID list' line
# id_val_str = lines[2].split(': ')[1].strip('[]\n')
# IDs_valCYC = [float(x) for x in id_val_str.split(', ')] if id_val_str else []  # The 'ID of cyclones in validation set' line
# print('IDs of the validation cyclones:\n',MOT, '\t',IDs_valCYC)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [5]:
ccc = 0
datesss=[]
for ids in [IDs_valCYC[-1]]:          # MAX
    ## cappear contains all indices of rows in d that have ids (one of the validation IDs)        
    cappear, = np.where((d[:,1]==ids))
    for wr,u in enumerate(cappear):
        tmp = d[u]
        k = str(helper.datenum_to_datetime(tmp[0]/24))
        Date = k[0:4]+k[5:7]+k[8:10]+'_'+k[11:13]
        ccc +=1
        datesss.append(Date)
    break

In [14]:
LAT = np.round(np.linspace(0,90,226),1)
LON = np.round(np.linspace(-180,180,901),1)


In [8]:
LAT[np.where(LAT==np.round(tmp[3],1))[0][0]]

54.4

In [13]:
LAT[np.where(LAT==np.round(tmp[3],1))]

array([54.4])

In [15]:
np.where(LON==np.round(tmp[2],1))[0][0]

480

In [16]:
addlon = CLONIDS + np.where(LON==np.round(tmp[2],1))[0][0]
addlon[np.where((addlon-900)>0)] = addlon[np.where((addlon-900)>0)]-900

addlon

12.0

---
### Structure of d:
-  0 col:    Time
-  1 col:    ID
-  2 col:    LON
-  3 col:    LAT
-  4 col:    Distance
-  5 col:    Area
-  6 col:    Inpres
-  7 col:    Outpres
-  8 col:    baroclin
-  9 col:    meanth
- 10 col:    ctype
- 11 col:    bergeron
- 12 col:    bmaxdiff
---

In [5]:
#--------------------------------------------------------------------
## I think this is stuff ALex only needed for mediteranean cyclones
# LATM = np.array([30,42,30,48])
# LONM = np.array([-5,2,42])

# #LONB = np.array([-105,-75,40,90])
# LONB = np.array([-95,-75,-5,15])
# LATB = np.array([15,83,0,83,50,83])

# LATSS = [LATM, LATB]
# LONSS = [LONM,LONB]
#---------------------------------------------------------------------
#r200_lonids, r200_latids = helper.radial_ids_around_center(200)

datastruct = dict()
#llat = len(r200_lonids)

labels = np.array([])
Gdates = np.array([])
data = dict()
###  restrict myself to Atlantic, tropical and Mediterranean ones



MONTHS = np.array(['DEC17','JAN18','FEB18','MAR18','APR18','MAY18','JUN18','JUL18','AUG18','SEP18','OCT18','NOV18'])
MONTHN = np.append(12,np.arange(1,12))


# # for at least 48 h backward trajectories require mature stage to be after the following dates
# mx = 3

MOT = 'DEC17'

months=[MOT]

LON = np.round(np.linspace(-180,180,901),1)
LAT = np.round(np.linspace(0,90,226),1)

ft = date.toordinal(date(1950,1,1))

add = 'All-CYC-entire-year-maxf' + MOT + '-correct'

datastruct = dict()

labels = np.array([])
Gdates = np.array([])
data = dict()




###  restrict myself to Atlantic, tropical and Mediterranean ones
for Month in months:
    datastruct[Month] = dict()
              
    print(f'\nSTART IDENTIFICATION OF MONTH: {Month}\n------------------------------\n')


    ## LOAD DATA
    ################################# MODIFIED BY MAX ############################################################################
    # - Load my Data and my validation IDs 
    path_trackCYC = f'/net/helium/atmosdyn/IFS-1Y/{Month}/features/tracking/TRACKED_CYCLONES'
    d = np.loadtxt(path_trackCYC,skiprows=1)
    IDs = np.unique(d[:,1])
    print(f'IDs from the TRACKED_CYCLONE file:\n{IDs}')

    ## Load validation cyclone IDs
    path_CYCsplit_info = f'/net/helium/atmosdyn/freimax/data_msc/IFS-18/cyclones/data_random_forest/{MOT}/cyclone_split_info.txt'
    with open(path_CYCsplit_info, 'r') as f:
        lines = f.readlines()
    num_cyclones = int(lines[0].split(': ')[1])  # The 'Number of cyclones' line
    id_list_str = lines[1].split(': ')[1].strip('[]\n')
    id_list_cyc = [float(x) for x in id_list_str.split(', ')] if id_list_str else []  # The 'ID list' line
    id_val_str = lines[2].split(': ')[1].strip('[]\n')
    IDs_valCYC = [float(x) for x in id_val_str.split(', ')] if id_val_str else []  # The 'ID of cyclones in validation set' line
    print(f'IDs of the validation cyclones:\n{MOT}\t{IDs_valCYC}\n')
    ################################################################################################################################



    # for ids in IDs[:]:            # ALEX
    for ids in IDs_valCYC:          # MAX

        print('Compute on CYC_id: ', ids)

        ## cappear contains all indices of rows in d that have ids (one of the validation IDs)        
        cappear, = np.where((d[:,1]==ids))


        clat = []
        clon = []
        hourszeta = np.array([])
        hourstoSLPmin = np.array([])
        zetal = np.array([])
        dates = np.array([])
        zeta = np.zeros(len(cappear))
        hours = np.zeros(len(cappear))
        
        
        hoursslp = np.array([])         # MAX
        slpl = np.array([])             # MAX
        slp = np.zeros(len(cappear))    # MAX


        ### find id of minimum SLP in track file
        slpminid = cappear[np.where(d[cappear,6] == np.min(d[cappear,6]))]
        slpminid = slpminid[-1]
        hourstoSLPmin = np.append(hourstoSLPmin, d[cappear,0]-d[slpminid,0])


        ## FROM MAX: Here the cappear file column 4 (5th column) is not correct
        # ### find id of minimum SLP in track file
        # slpminid = cappear[np.where(d[cappear,4] == np.min(d[cappear,4]))]      # NOTE: d[cappear,4] IS EQUAL TO d[:,4]   --> values of 5th column = "inpres"
        # slpminid = slpminid[-1]                                                 # Extract number from array
        # hourstoSLPmin = np.append(hourstoSLPmin, d[cappear,0]-d[slpminid,0])    # Substract time from min SLP from other times to get seconds to/from SLP minimum



        ### step through hrs of the cyclon in trackfile
        for wr,u in enumerate(cappear):
            tmp = d[u]
            k = str(helper.datenum_to_datetime(ft+tmp[0]/24))
            Date = k[0:4]+k[5:7]+k[8:10]+'_'+k[11:13]


            ### if that date has not been loaded yet, load it
            # if Gdates.size == 0 or (~(np.any(Gdates==Date))):     # MAX
            if (~(np.any(Gdates==Date))):                           # ALEX
                # Load dataset with PS, SLP and Vorticity
                dataset_1 = xr.open_dataset(f'/net/helium/atmosdyn/IFS-1Y/{Month}/cdf/S{Date}', drop_variables=['PV','P','TH','THE','RH','PVRCONVT','PVRCONVM','PVRTURBT','PVRTURBM','PVRLS','PVRCOND','PVRSW','PVRLWH','PVRLWC','PVRDEP','PVREVC','PVREVR','PVRSUBI','PVRSUBS','PVRMELTI','PVRMELTS','PVRFRZ','PVRRIME','PVRBF'])
                data[Date] = dataset_1
            else:
                 print('\nIMPRTANT SECTION WAS LEFT OUT\n')
            
            dates = np.append(dates,Date)
            Gdates = np.append(Gdates,Date)
                
                ####################################################



            #hours since first identification
            hours[wr] = tmp[0]
            ### keep adding lon to lon and lat to lat and use srtid, as it is correct with normal crosssections
            CLONIDS, CLATIDS = helper.IFS_radial_ids_correct(200,LAT[np.where(LAT==np.round(tmp[3],1))[0][0]])
            addlon = CLONIDS + np.where(LON==np.round(tmp[2],1))[0][0]
            addlon[np.where((addlon-900)>0)] = addlon[np.where((addlon-900)>0)]-900

            clat.append(CLATIDS.astype(int) + np.where(LAT==np.round(tmp[3],1))[0][0])
            clon.append(addlon.astype(int))
#                 clat = np.vstack((clat, CLATIDS + np.where(LAT==np.round(tmp[3],1))[0][0]))
#                 clon = np.vstack((clon, addlon))

            ### use last lon and lat entries [-1] of center
            PS = data[Date].PS.values[0,0,clat[-1],clon[-1]]
            for e in range(len(CLATIDS)):
                P = helper.modellevel_to_pressure(PS[e])
                I = np.where(abs(P-850)==np.min(abs(P-850)))[0]
                I = I[0].astype(int)
                zetal = np.append(zetal,data[Date].VORT.values[0,I,clat[-1][e],clon[-1][e]])

            zeta[wr] = np.mean(zetal)

        hours = hours - hours[np.where(zeta==np.max(zeta))]
        hourszeta = np.append(hourszeta,hours)

        # allow for at least 48h backward trajectories
        if (((int(dates[np.where(zeta==np.max(zeta))[0][-1]][6:8])>=3)) | (int(dates[np.where(zeta==np.max(zeta))[0][-1]][4:6])!=MONTHN[np.where(MONTHS==MOT)[0][0]])):

            ### added 30.10.2020 to ensure only MED cyclones
            ### add 1 as clat, clon start with zeros as base to stack them

            MatureLat = np.round(np.mean(LAT[clat[np.where(zeta==np.max(zeta))[0][-1]]]),1)
            MatureLon = np.round(np.mean(LON[clon[np.where(zeta==np.max(zeta))[0][-1]]]),1)
            # use latest maximum relative vorticity
    #              MatureLat = np.round(np.mean(LAT[clat[np.where(zeta==np.max(zeta))[0][-1]+1]]),1)
    #              MatureLon = np.round(np.mean(LON[clon[np.where(zeta==np.max(zeta))[0][-1]+1]]),1)
        
        #if time to mature stage is too short remove from data
        # if((hours[0]<=(-2)) & (hours[-1]>=2)):        # ALEX
        if((hours[0]<=(-0)) & (hours[-1]>=0)):          # MAX
            datastruct[Month][ids] = dict()
            datastruct[Month][ids]['clat'] = clat
            datastruct[Month][ids]['clon'] = clon
            datastruct[Month][ids]['zeta'] = zeta
            datastruct[Month][ids]['hzeta'] = hourszeta
            datastruct[Month][ids]['SLP'] = d[cappear,6]
            datastruct[Month][ids]['hSLP'] = hourstoSLPmin
            datastruct[Month][ids]['dates'] = dates
            datastruct[Month][ids]['Matlat'] = MatureLat
            datastruct[Month][ids]['Matlon'] = MatureLon
            # ### these are preidentified tropical
            # if(d[cappear[0],-3]==0):
            #         label=0
            # ### MED
            # else:
            #     for b,o in enumerate(LONM[:-1]):
            #         if( ((30-MatureLat)<=0) & ((LATM[b]-MatureLat)>=0) & ((o-MatureLon)<=0) & ((LONM[b+1]-MatureLon)>=0)):
            #                 label=2
            # ### ATL
            #         else:
            #             label=1
            datastruct[Month][ids]['label'] = ids
            print('Saved cyclone ID:', ids)

        else: 
            print(f'File not saved, becuae time to mature stage is not long enough: {hours[0]}, {hours[-1]} ')

PATH = '/net/helium/atmosdyn/freimax/data_msc/IFS-18/IFS-traj/CYC_validation'

# Create the directory if it does not exist
os.makedirs(PATH + '/' + MOT, exist_ok=True)

f = open(PATH + '/' + MOT + '/' + add + '.txt','wb')
pickle.dump(datastruct,f)
f.close()


START IDENTIFICATION OF MONTH: DEC17
------------------------------

IDs from the TRACKED_CYCLONE file:
[ 16.  30.  33.  38.  42.  52.  53.  55.  66.  69.  70.  74.  75.  77.
  78.  79.  85. 102. 103. 111. 116. 118. 119. 123.]
IDs of the validation cyclones:
DEC17	[111.0, 33.0]

Compute on CYC_id:  111.0


/tmp/ipykernel_53770/1818112968.py:128: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (~(np.any(Gdates==Date))):                           # ALEX


Saved cyclone ID: 111.0
Compute on CYC_id:  33.0
Saved cyclone ID: 33.0


In [9]:
Date

'20171212_02'

In [15]:
Gdates1 = np.array([])
if Gdates1.size != 0 or (~(np.any(Gdates1==Date))):   # MAX
    print('jup')

jup


/tmp/ipykernel_40608/561977917.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if Gdates1.size != 0 or (~(np.any(Gdates1==Date))):   # MAX


---
### Max tries stuff here

In [12]:
track = '/net/helium/atmosdyn/freimax/data_msc/casestudy_ra/data/TRACKED_CYCLONE.txt'
d = np.loadtxt(track, skiprows=1)


# d is a list:            time  |  ?  |  lon  |  lat  | SLP-min

In [13]:
d[:,1]
d[1:5,:]

array([[1.49178600e+09, 0.00000000e+00, 1.58000000e+02, 4.55999980e+01,
        9.84758118e+02],
       [1.49178960e+09, 0.00000000e+00, 1.58399994e+02, 4.55999980e+01,
        9.82932495e+02],
       [1.49179320e+09, 0.00000000e+00, 1.58399994e+02, 4.60000000e+01,
        9.81590576e+02],
       [1.49179680e+09, 0.00000000e+00, 1.58800003e+02, 4.60000000e+01,
        9.79677490e+02]])

In [14]:
cappear, = np.where((d[:,1]==d[:,1][0]))
cappear

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36])

In [15]:
slpminid = cappear[np.where(d[cappear,4] == np.min(d[cappear,4]))]      # NOTE: d[cappear,4] IS EQUAL TO d[:,4]   --> values of 5th column = "inpres"
slpminid = slpminid[-1]
slpminid

17

In [16]:
hourstoSLPmin = np.array([])
hourstoSLPmin = np.append(hourstoSLPmin, d[cappear,0]-d[slpminid,0])

hourstoSLPmin

array([-61200., -57600., -54000., -50400., -46800., -43200., -39600.,
       -36000., -32400., -28800., -25200., -21600., -18000., -14400.,
       -10800.,  -7200.,  -3600.,      0.,   3600.,   7200.,  10800.,
        14400.,  18000.,  21600.,  25200.,  28800.,  32400.,  36000.,
        39600.,  43200.,  46800.,  50400.,  54000.,  57600.,  61200.,
        64800.,  68400.])

In [359]:
# parser = argparse.ArgumentParser(description="produce txt files with identification:")
# parser.add_argument('months',default='',type=str,help='month combination like DJF')

# args = parser.parse_args()
# MOT = str(args.months)



############### ADDED BY MAX #######################
MOT = 'APR17'       # WE CANNOT USE ARGPARSE I IPYNB
####################################################


MONTHS = np.array(['DEC17','JAN18','FEB18','MAR17','APR17','MAY18','JUN18','JUL18','AUG18','SEP18','OCT18','NOV18'])
MONTHN = np.append(12,np.arange(1,12))

months=[MOT]

LON = np.round(np.linspace(-180,180,901),1)
LAT = np.round(np.linspace(0,90,226),1)


ft = date.toordinal(date(1950,1,1))

add = 'CYC-CaseStudy' + MOT + '-correct'
#r200_lonids, r200_latids = helper.radial_ids_around_center(200)

datastruct = dict()
#llat = len(r200_lonids)

labels = np.array([])
Gdates = np.array([])
data = dict()

IDs = np.unique(d[:,1])
for ids in IDs[:]:
    print(type(ids))
    IDs = ids




for Month in months:
    datastruct[Month] = dict()

    ## LOAD DATA
    ### added by MAX-----------------------------------------------------------------------------------
    print(f'Case study cyclone {Month} added (Max)')
    track = '/net/helium/atmosdyn/freimax/data_msc/casestudy_ra/data/TRACKED_CYCLONE.txt'
    d = np.loadtxt(track, skiprows=1)
    #-------------------------------------------------------------------------------------------------


    # Return an array with all indices that satisfy d[:,1]==d[:,1][0]  -> Return an array with ALL indices (from 0 to 36)
    cappear, = np.where((d[:,1]==d[:,1][0]))
    
    clat = []
    clon = []
    hourstoSLPmin = np.array([])
    dates = np.array([])
    hours = np.zeros(len(cappear))
    
    # hourszeta = np.array([])
    # zetal = np.array([])
    # zeta = np.zeros(len(cappear))
    
    hoursslp = np.array([])
    slpl = np.array([])
    slp = np.zeros(len(cappear))


    ### find id of minimum SLP in track file
    slpminid = cappear[np.where(d[cappear,4] == np.min(d[cappear,4]))]      # NOTE: d[cappear,4] IS EQUAL TO d[:,4]   --> values of 5th column = "inpres"
    slpminid = slpminid[-1]                                                 # Extract number from array
    
    hourstoSLPmin = np.append(hourstoSLPmin, d[cappear,0]-d[slpminid,0])    # Substract time from min SLP from other times to get seconds to/from SLP minimum


    ### step through hrs of the cyclon in trackfile
    for wr,u in enumerate(cappear):

        tmp = d[u]
        k = str(datetime.utcfromtimestamp(tmp[0]))
        Date = k[0:4]+k[5:7]+k[8:10]+'_'+k[11:13]


        ### if that date has not been loaded yet, load it
        if Gdates.size == 0 or (~(np.any(Gdates==Date))):

            #### Date generation from datetime floats (Max) ####
            # Only retain Q, RWC, LWC, IWC, SWC, CC, T, OMEGA, U, V, SLP, PS, tsubi, tevr, tsubs, tmelti, tmelts
            dataset_1 = xr.open_dataset('/net/thermo/atmosdyn2/atroman/PACaccu/cdf/P'+Date, drop_variables=['ttot','tdyn','tsw','tlw','tmix','tconv','tls','tcond','tdep','tbf','tevc','tfrz','trime','tce'])
            data[Date] = dataset_1
            dates = np.append(dates,Date)
            Gdates = np.append(Gdates,Date)
            ####################################################


            # seconds since first identification
            hours[wr] = tmp[0]      # hours was initalized as np.zeros(len(cappear))


            ######################################## RADIUS AROUND SLP-MIN DO NOT UDNERSTADN HOW YET #################################
            ### keep adding lon to lon and lat to lat and use srtid, as it is correct with normal crosssections
            CLONIDS, CLATIDS = helper.IFS_radial_ids_correct(200, LAT[np.where(LAT==np.round(tmp[3],1))[0][0]] )
            addlon = CLONIDS + np.where(LON==np.round(tmp[2],1))[0][0]
            addlon[np.where((addlon-900)>0)] = addlon[np.where((addlon-900)>0)]-900
            clat.append(CLATIDS.astype(int) + np.where(LAT==np.round(tmp[3],1))[0][0])
            clon.append(addlon.astype(int))
            #------------------------------------------------------------------------------------------------------------------------
            

                # for e in range(len(CLATIDS)):
                #     P = helper.modellevel_to_pressure(PS[e])
                #     I = np.where(abs(P-850)==np.min(abs(P-850)))[0]
                #     I = I[0].astype(int)
                #     zetal = np.append(zetal,data[Date].VORT.values[0,I,clat[-1][e],clon[-1][e]])


    
            slpl = np.append(slpl, data[Date].SLP.isel(lat=slice(clat[-1][0]+225,clat[-1][-1]+225) , lon=slice(clon[-1][0],clon[-1][-1])).values)
            
            slp[wr] = np.min(slpl)

            min_slp_t = data[Date].SLP.isel(lat=slice(clat[-1][0]+225,clat[-1][-1]+225) , lon=slice(clon[-1][0],clon[-1][-1])).values.min()
            print(f'{str(data[Date].time.values)[0:18]}\t\tMin slp: {min_slp_t:.1f}\tLonbox: [{data[Date].lon[clon[-1].min()].values:.1f}, {data[Date].lon[clon[-1].max()].values:.1f}]  |  Latbox: [{data[Date].lat[clat[-1].min()+225].values:.1f}, {data[Date].lat[(clat[-1]+225).max()].values:.1f}]')



    # hours = hours - hours[np.where(slp==np.min(slp))]
    # hoursslp = np.append(hoursslp,hours)
    
    
    # if  ( (int( dates[np.where(zeta==np.max(zeta))[0][-1]][6:8] ) >=3 ) ) | (int(dates[np.where(zeta==np.max(zeta))[0][-1]][4:6])!=MONTHN[np.where(MONTHS==MOT)[0][0]] ):
        ### add 1 as clat, clon start with zeros as base to stack them
        # MatureLat = np.round(np.mean(LAT[clat[np.where(zeta==np.max(zeta))[0][-1]]]),1)
        # MatureLon = np.round(np.mean(LON[clon[np.where(zeta==np.max(zeta))[0][-1]]]),1)

    if  ( (int( dates[np.where(slp==np.min(slp))[0][-1]][6:8] ) >=3 ) ) | (int(dates[np.where(slp==np.min(slp))[0][-1]][4:6])!=MONTHN[np.where(MONTHS==MOT)[0][0]] ):        # THIS CONDTION MAKES NO SENSE FOR MEEE?????!!!!!

        ### add 1 as clat, clon star            print(f'') with zeros as base to stack them
        MatureLat = np.round(np.mean(LAT[clat[np.where(slp==np.min(slp))[0][-1]]]),1)
        MatureLon = np.round(np.mean(LON[clon[np.where(slp==np.min(slp))[0][-1]]]),1)
        print(MatureLat, MatureLon)
    

        if True:
            datastruct[Month][IDs] = dict()
            datastruct[Month][IDs]['clat'] = clat
            datastruct[Month][IDs]['clon'] = clon
            datastruct[Month][IDs]['zeta'] = slp            #zeta
            # datastruct[Month][IDs]['hzeta'] = hourszeta     
            datastruct[Month][IDs]['SLP'] = d[cappear,4]
            datastruct[Month][IDs]['hSLP'] = hourstoSLPmin
            datastruct[Month][IDs]['dates'] = dates
            datastruct[Month][IDs]['Matlat'] = MatureLat
            datastruct[Month][IDs]['Matlon'] = MatureLon

        # ### these are preidentified tropical
        # if(d[cappear[0],-3]==0):
        #         label=0
        # ### MED
        # else:
        #     for b,o in enumerate(LONM[:-1]):
        #         if( ((30-MatureLat)<=0) & ((LATM[b]-MatureLat)>=0) &
        #     ((o-MatureLon)<=0) & ((LONM[b+1]-MatureLon)>=0)):
        #                 label=2
        # ### ATL
        #         else:
        #             label=1

        ### Max set the label to something
        label = 0. 




        datastruct[Month][IDs]['label'] = label


PATH = '/net/helium/atmosdyn/freimax/data_msc/casestudy_ra/data/features/'
full_path = os.path.join(PATH, MOT, add + '.txt')

# Create directories if they do not exist
os.makedirs(os.path.dirname(full_path), exist_ok=True)

# Now you can write the file
with open(full_path, 'wb') as f:
    pickle.dump(datastruct, f)


<class 'numpy.float64'>
Case study cyclone APR17 added (Max)
['2017-04-10T00:00		Min slp: 985.8	Lonbox: [155.2, 160.0]  |  Latbox: [43.2, 46.4]
['2017-04-10T01:00		Min slp: 984.8	Lonbox: [155.2, 160.0]  |  Latbox: [43.6, 46.8]
['2017-04-10T02:00		Min slp: 982.9	Lonbox: [155.6, 160.4]  |  Latbox: [43.6, 46.8]
['2017-04-10T03:00		Min slp: 981.6	Lonbox: [155.6, 160.4]  |  Latbox: [44.0, 47.2]
['2017-04-10T04:00		Min slp: 979.7	Lonbox: [156.0, 160.8]  |  Latbox: [44.0, 47.2]
['2017-04-10T05:00		Min slp: 978.1	Lonbox: [156.0, 160.8]  |  Latbox: [44.0, 47.2]
['2017-04-10T06:00		Min slp: 976.4	Lonbox: [156.4, 161.2]  |  Latbox: [44.4, 47.6]
['2017-04-10T07:00		Min slp: 974.7	Lonbox: [156.4, 161.2]  |  Latbox: [44.4, 47.6]
['2017-04-10T08:00		Min slp: 973.4	Lonbox: [156.8, 161.6]  |  Latbox: [44.4, 47.6]
['2017-04-10T09:00		Min slp: 972.2	Lonbox: [156.8, 161.6]  |  Latbox: [44.4, 47.6]
['2017-04-10T10:00		Min slp: 971.3	Lonbox: [157.2, 162.0]  |  Latbox: [44.4, 47.6]
['2017-04-10T11:00		Min sl

'All-CYC-entire-year-NEWAPR17-correct'

#### Alex script continues here
---

In [8]:
###  restrict myself to Atlantic, tropical and Mediterranean ones
for Month in months:
    datastruct[Month] = dict()

    if Month == 'DEC17':
        track = dbase + '002-2020-08-05-Identify-DJF1718-medcyclones/' + Month  + '/TRACKED_CYCLONES'
        d = np.loadtxt(track)

    ### added by MAX
    elif Month == 'APR17':
        track = '/net/helium/atmosdyn/freimax/data_msc/casestudy_ra/data/features/TRACKED_CYCLONE.txt'
        d = np.loadtxt(track, skiprows=1)
    else:
        track = '/home/atroman/phd/' + Month + '/features/tracking/TRACKED_CYCLONES'
        d = np.loadtxt(track,skiprows=1)

    IDs = np.unique(d[:,1])
    for ids in IDs[:]:
       cappear, = np.where((d[:,1]==ids))
       indomainalready = 0
       for lonb, latb in zip(LONSS,LATSS):
        if indomainalready==1:
            continue
        #first & last & mid track point in boundary

        if((d[cappear[0],2]>lonb[0]) & (d[cappear[0],2]<lonb[-1])):

         if ((d[cappear[-1],2]>lonb[0]) & (d[cappear[-1],2]<lonb[-1])):

          if ((d[cappear[int(len(cappear)/2)],2]>lonb[0]) & (d[cappear[int(len(cappear)/2)],2]<lonb[-1])):
           CONTINUE=0

           # check lat boundary
           for k,l in enumerate(lonb[:-1]):
                if ((d[cappear[0],2]>l) & (d[cappear[0],2]<lonb[k+1]) & (d[cappear[0],3]>latb[2*k]) &(d[cappear[0],3]<latb[2*k+1])):
                    CONTINUE=1
                    indomainalready=1
           if (CONTINUE==1):
            #all local data for particular cyclones
#            clat = np.zeros((len(r200_lonids)),dtype=int)
#            clon = np.zeros((len(r200_lonids)),dtype=int)
            clat = []
            clon = []
            hourszeta = np.array([])
            hourstoSLPmin = np.array([])
            zetal = np.array([])
            dates = np.array([])
            zeta = np.zeros(len(cappear))
            hours = np.zeros(len(cappear))

            ### find id of minimum SLP in track file
            slpminid = cappear[np.where(d[cappear,6] == np.min(d[cappear,6]))]
            slpminid = slpminid[-1]
            hourstoSLPmin = np.append(hourstoSLPmin, d[cappear,0]-d[slpminid,0])

            ### step through hrs of the cyclon in trackfile
            for wr,u in enumerate(cappear):
                tmp = d[u]
                k = str(helper.datenum_to_datetime(ft+tmp[0]/24))
                Date = k[0:4]+k[5:7]+k[8:10]+'_'+k[11:13]

                ### if that date has not been loaded yet, load it
                if (~(np.any(Gdates==Date))):
                    data[Date] = xr.open_dataset('/net/thermo/atmosdyn/atroman/phd/'+Month+'/cdf/S'+Date, drop_variables=['PV','P','TH','THE','RH','PVRCONVT','PVRCONVM','PVRTURBT','PVRTURBM','PVRLS','PVRCOND','PVRSW','PVRLWH','PVRLWC','PVRDEP','PVREVC','PVREVR','PVRSUBI','PVRSUBS','PVRMELTI','PVRMELTS','PVRFRZ','PVRRIME','PVRBF'])

                dates = np.append(dates,Date)
                Gdates = np.append(Gdates,Date)

                #hours since first identification
                hours[wr] = tmp[0]
                ### keep adding lon to lon and lat to lat and use srtid, as it is correct with normal crosssections
                CLONIDS, CLATIDS = helper.IFS_radial_ids_correct(200,LAT[np.where(LAT==np.round(tmp[3],1))[0][0]])
                addlon = CLONIDS + np.where(LON==np.round(tmp[2],1))[0][0]
                addlon[np.where((addlon-900)>0)] = addlon[np.where((addlon-900)>0)]-900

                clat.append(CLATIDS.astype(int) + np.where(LAT==np.round(tmp[3],1))[0][0])
                clon.append(addlon.astype(int))
#                clat = np.vstack((clat, CLATIDS + np.where(LAT==np.round(tmp[3],1))[0][0]))
#                clon = np.vstack((clon, addlon))

                ### use last lon and lat entries [-1] of center
                PS = data[Date].PS.values[0,0,clat[-1],clon[-1]]
                for e in range(len(CLATIDS)):
                    P = helper.modellevel_to_pressure(PS[e])
                    I = np.where(abs(P-850)==np.min(abs(P-850)))[0]
                    I = I[0].astype(int)
                    zetal = np.append(zetal,data[Date].VORT.values[0,I,clat[-1][e],clon[-1][e]])

                zeta[wr] = np.mean(zetal)

            hours = hours - hours[np.where(zeta==np.max(zeta))]
            hourszeta = np.append(hourszeta,hours)

            # allow for at least 48h backward trajectories
            if (((int(dates[np.where(zeta==np.max(zeta))[0][-1]][6:8])>=3)) | (int(dates[np.where(zeta==np.max(zeta))[0][-1]][4:6])!=MONTHN[np.where(MONTHS==MOT)[0][0]])):

            ### added 30.10.2020 to ensure only MED cyclones
            ### add 1 as clat, clon start with zeros as base to stack them

             MatureLat = np.round(np.mean(LAT[clat[np.where(zeta==np.max(zeta))[0][-1]]]),1)
             MatureLon = np.round(np.mean(LON[clon[np.where(zeta==np.max(zeta))[0][-1]]]),1)
            # use latest maximum relative vorticity
#             MatureLat = np.round(np.mean(LAT[clat[np.where(zeta==np.max(zeta))[0][-1]+1]]),1)
#             MatureLon = np.round(np.mean(LON[clon[np.where(zeta==np.max(zeta))[0][-1]+1]]),1)
            #if time to mature stage is too short remove from data
             if((hours[0]<=(-2)) & (hours[-1]>=2)):
                datastruct[Month][ids] = dict()
                datastruct[Month][ids]['clat'] = clat
                datastruct[Month][ids]['clon'] = clon
                datastruct[Month][ids]['zeta'] = zeta
                datastruct[Month][ids]['hzeta'] = hourszeta
                datastruct[Month][ids]['SLP'] = d[cappear,6]
                datastruct[Month][ids]['hSLP'] = hourstoSLPmin
                datastruct[Month][ids]['dates'] = dates
                datastruct[Month][ids]['Matlat'] = MatureLat
                datastruct[Month][ids]['Matlon'] = MatureLon

                ### these are preidentified tropical
                if(d[cappear[0],-3]==0):
                     label=0
                ### MED
                else:
                    for b,o in enumerate(LONM[:-1]):
                        if( ((30-MatureLat)<=0) & ((LATM[b]-MatureLat)>=0) &
                   ((o-MatureLon)<=0) & ((LONM[b+1]-MatureLon)>=0)):
                             label=2
                ### ATL
                        else:
                         label=1
                datastruct[Month][ids]['label'] = label

# f = open(PATH + MOT + '/' + add + '.txt','wb')
# pickle.dump(datastruct,f)
# f.close()

/tmp/ipykernel_23599/2250215231.py:63: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (~(np.any(Gdates==Date))):
